In [6]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

# Cazoo

## Testing

In [45]:
url = 'https://www.cazoo.co.uk/cars/'
res = requests.get(url)
res

<Response [200]>

In [46]:
soup = BeautifulSoup(res.content, 'html.parser')
table = soup.find(attrs={'class':"resultsstyles__ResultsList-sc-11kjmxu-0 ezjHRs"})
tbody = table.find_all('li')
tr = tbody[2]
make_model = tr.find(attrs={'class':'vehicle-cardstyles__CustomTitle-sc-1bxv5iu-8 vEuQf'}).text
version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
dets= tr.find_all(attrs={'data-test-id':'tags'})
version = version.split()
engine = ''.join(version[0])
trim = " ".join(version[1:])
price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text

In [47]:
split = make_model.split()
split[0]

'Skoda'

## Automating

In [32]:
manufactures = {'abarth', 'alfa-romeo', 'audi', 'bmw', 'citroen', 'cupra', 'dacia', 'ds', 'fiat',
               'ford', 'honda', 'hyundai','jaguar', 'jeep', 'kia', 'land-rover',
               'lexus', 'mazda', 'mg','mini','mitsubishi','nissan','peugeot',
               'polestar','renault','seat','skoda','smart','subaru','suzuki','tesla',
               'toyota','vauxhall','volkswagen','volvo'}

In [33]:
vehicles = []
for make in manufactures:
    url = 'https://www.cazoo.co.uk/cars/' + make
    print(make)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    table = soup.find(attrs={'data-test-id':"results-list"})
    tbody = table.find_all('li')
    tbody = tbody[2:]
    for tr in tbody:
        make_model = tr.find(attrs={'data-test-id':'card-title'}).text
        makemod = make_model.split()
        make = makemod[0]
        model = ' '.join(makemod[1:])
        version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
        version = version.split()
        engine = ''.join(version[0])
        trim = " ".join(version[1:])
        dets = tr.find_all(attrs={'data-test-id':'tags'})
        details = dets[0]
        details_str = str(details)  # Convert the first object in the list to a string
        matches = re.findall(r'(\d[\d,]* miles|\d{4} reg|Auto|Manual|Petrol|Diesel|Hybrid|\
        Electric)', details_str)
        trans = matches[2] if 'Auto' in matches[2] or 'Manual' in matches[2] else None
        mileage = matches[0] if 'miles' in matches[0] else None
        registration = matches[1] if 'reg' in matches[1] else None
       
        price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text
        vehicle = {'make':make, 'model': model, 'engine_size':engine,'trim':trim, 'mileage':mileage,
               'reg_year':registration, 'transmission_type':trans, 'fuel_type':fuel_type,
               'price':price
              }
        vehicles.append(vehicle)

vauxhall
skoda
bmw
ford
honda
audi
suzuki
citroen
mitsubishi
lexus
renault
subaru
mazda
mini
nissan
alfa-romeo
tesla
dacia
hyundai
mg
volkswagen
peugeot
ds
jeep
kia
jaguar
abarth
polestar
volvo
land-rover
fiat
cupra
smart
seat
toyota


In [82]:
p1s = pd.DataFrame(vehicles)
p1s


,make,model,engine_size,trim,mileage,reg_year,transmission_type,fuel_type,price
0,Vauxhall,Corsa,1.4i,ecoTEC Design,"77,055 miles",2019 reg,Manual,Petrol,"£6,850"
1,Vauxhall,Corsa,1.4i,ecoTEC Griffin,"68,065 miles",2019 reg,Manual,Petrol,"£7,500"
2,Vauxhall,Corsa,1.4i,ecoTEC Design,"69,292 miles",2019 reg,Manual,Petrol,"£7,400"
3,Vauxhall,Corsa,1.4i,ecoTEC Energy,"49,544 miles",2018 reg,Manual,Petrol,"£7,950"
4,Vauxhall,Viva,1.0i,SE,"26,255 miles",2018 reg,Manual,Petrol,"£7,850"
...,...,...,...,...,...,...,...,...,...
1201,Toyota,Yaris,1.5,VVT-h Icon E-CVT,"47,622 miles",2021 reg,Auto,Petrol,"£13,500"
1202,Toyota,Yaris,1.5,VVT-h Icon Tech E-CVT,"23,911 miles",2018 reg,Auto,Petrol,"£13,250"
1203,Toyota,Yaris,1.5,VVT-h Icon E-CVT,"48,638 miles",2021 reg,Auto,Petrol,"£13,550"
1204,Toyota,AYGO,1.0,VVT-i x-trend x-shift (Safety Sense),"4,507 miles",2021 reg,Auto,Petrol,"£13,550"


In [43]:
pagesvehicles = []
for make in manufactures:
    url = 'https://www.cazoo.co.uk/cars/' + make
    for i in range(2, 13):
        url = url + '/?page=' + str(i)
        print(make, i)
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'html.parser')
        table = soup.find(attrs={'data-test-id':"results-list"})
        if table != None:
            tbody = table.find_all('li')
            tbody = tbody[2:]
            for tr in tbody:
                make_model = tr.find(attrs={'data-test-id':'card-title'}).text
                makemod = make_model.split()
                make = makemod[0]
                model = ' '.join(makemod[1:])
                version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
                version = version.split()
                engine = ''.join(version[0])
                trim = " ".join(version[1:])
                dets = tr.find_all(attrs={'data-test-id':'tags'})
                details = dets[0]
                details_str = str(details)  # Convert the first object in the list to a string
                matches = re.findall(r'(\d[\d,]* miles|\d{4} reg|Auto|Manual|Petrol|Diesel|Hybrid|\
                Electric)', details_str)
                trans = matches[2] if 'Auto' in matches[2] or 'Manual' in matches[2] else None
                mileage = matches[0] if 'miles' in matches[0] else None
                registration = matches[1] if 'reg' in matches[1] else None
       
                price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text
                vehicle = {'make':make, 'model': model, 'engine_size':engine,'trim':trim, 'mileage':mileage,
               'reg_year':registration, 'transmission_type':trans, 'fuel_type':fuel_type,
               'price':price
              }
                pagesvehicles.append(vehicle)

vauxhall 2
Vauxhall 3
Vauxhall 4
Vauxhall 5
Vauxhall 6
Vauxhall 7
Vauxhall 8
Vauxhall 9
Vauxhall 10
Vauxhall 11
Vauxhall 12
skoda 2
Skoda 3
Skoda 4
Skoda 5
Skoda 6
Skoda 7
Skoda 8
Skoda 9
Skoda 10
Skoda 11
Skoda 12
bmw 2
BMW 3
BMW 4
BMW 5
BMW 6
BMW 7
BMW 8
BMW 9
BMW 10
BMW 11
BMW 12
ford 2
Ford 3
Ford 4
Ford 5
Ford 6
Ford 7
Ford 8
Ford 9
Ford 10
Ford 11
Ford 12
honda 2
honda 3
honda 4
honda 5
honda 6
honda 7
honda 8
honda 9
honda 10
honda 11
honda 12
audi 2
Audi 3
Audi 4
Audi 5
Audi 6
Audi 7
Audi 8
Audi 9
Audi 10
Audi 11
Audi 12
suzuki 2
suzuki 3
suzuki 4
suzuki 5
suzuki 6
suzuki 7
suzuki 8
suzuki 9
suzuki 10
suzuki 11
suzuki 12
citroen 2
Citroen 3
Citroen 4
Citroen 5
Citroen 6
Citroen 7
Citroen 8
Citroen 9
Citroen 10
Citroen 11
Citroen 12
mitsubishi 2
mitsubishi 3
mitsubishi 4
mitsubishi 5
mitsubishi 6
mitsubishi 7
mitsubishi 8
mitsubishi 9
mitsubishi 10
mitsubishi 11
mitsubishi 12
lexus 2
lexus 3
lexus 4
lexus 5
lexus 6
lexus 7
lexus 8
lexus 9
lexus 10
lexus 11
lexus 12
renault 2
Ren

In [44]:
pages = pd.DataFrame(pagesvehicles)
pages

,make,model,engine_size,trim,mileage,reg_year,transmission_type,fuel_type,price
0,Vauxhall,Corsa,1.2,Turbo SRi Premium,"45,361 miles",2020 reg,Manual,Petrol,"£10,400"
1,Vauxhall,Crossland X,1.2,Turbo Griffin,"46,653 miles",2020 reg,Manual,Petrol,"was £10,150£9,950"
2,Vauxhall,Crossland X,1.2,Turbo Griffin,"46,935 miles",2020 reg,Manual,Petrol,"£9,950"
3,Vauxhall,Corsa,1.4i,ecoFLEX Energy (a/c),"42,886 miles",2015 reg,Manual,Petrol,"£7,650"
4,Vauxhall,Grandland X,1.5,Turbo D BlueInjection Tech Line Nav,"80,193 miles",2019 reg,Manual,Petrol,"£10,550"
...,...,...,...,...,...,...,...,...,...
9092,Toyota,Yaris,1.5,VVT-h Dynamic E-CVT,"39,167 miles",2020 reg,Auto,Petrol,"£15,850"
9093,Toyota,Yaris,1.5,VVT-h Excel E-CVT (15in Alloy),"39,442 miles",2017 reg,Auto,Petrol,"£12,000"
9094,Toyota,Corolla,1.8,VVT-h Icon Tech CVT,"13,220 miles",2020 reg,Auto,Petrol,"£17,050"
9095,Toyota,C-HR,1.8,VVT-h Excel CVT,"45,062 miles",2020 reg,Auto,Petrol,"£17,700"


## Mercs

## Merge

# Cinch

## Testing

In [63]:
url2 = 'https://www.cargurus.co.uk/Cars/inventorylisting/viewDetailsFilterViewInventoryListing.action?zip=CR0%207HL&inventorySearchWidgetType=AUTO&sortDir=ASC&sourceContext=carGurusHomePageModel&distance=50&sortType=DEAL_SCORE'
res2 = requests.get(url2)
res2

<Response [200]>

In [73]:
soup = BeautifulSoup(res2.content)
t1 = soup.find(attrs={'id':'main'})
t1.find_all(attrs={'data-testid':'srp-tile-listing-standard'})

[]

In [76]:
url = 'https://www.cargurus.co.uk/Cars/inventorylisting/viewDetailsFilterViewInventoryListing.action?zip=CR0%207HL&inventorySearchWidgetType=AUTO&sortDir=ASC&sourceContext=carGurusHomePageModel&distance=50&sortType=DEAL_SCORE'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0;Win64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}
response = requests.get(url, headers=headers)

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find the HTML elements containing the vehicle data
vehicle_listings = soup.find_all('div', class_='vehicle-card')

# Extract and print the relevant information for each vehicle listing
for listing in vehicle_listings:
    # Extract specific details from the listing
    title = listing.find('h4', class_='cg-dealFinder-result-title').text.strip()
    price = listing.find('span', class_='cg-dealFinder-result-price').text.strip()
    mileage = listing.find('div', class_='cg-dealFinder-result-stats').find_all('span')[1].text.strip()

    # Print the extracted information
    print(f"Title: {title}")
    print(f"Price: {price}")
    print(f"Mileage: {mileage}")
    print()

In [80]:
driver_path = 'C:\Program Files (x86)\Google'

In [81]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Set the path to your ChromeDriver executable
driver_path = 'path/to/chromedriver.exe'

# Set Chrome options
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run Chrome in headless mode

# Create a new ChromeDriver service
service = Service(driver_path)

# Create a new instance of the Chrome driver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Navigate to the website URL
url = 'https://www.cargurus.co.uk/Cars/inventorylisting/viewDetailsFilterViewInventoryListing.action?zip=CR0%207HL&inventorySearchWidgetType=AUTO&sortDir=ASC&sourceContext=carGurusHomePageModel&distance=50&sortType=DEAL_SCORE'
driver.get(url)

# Find the HTML elements containing the vehicle data
vehicle_listings = driver.find_elements(By.CLASS_NAME, 'vehicle-card')

# Extract and print the relevant information for each vehicle listing
for listing in vehicle_listings:
    # Extract specific details from the listing
    title = listing.find_element(By.CLASS_NAME, 'cg-dealFinder-result-title').text.strip()
    price = listing.find_element(By.CLASS_NAME, 'cg-dealFinder-result-price').text.strip()
    mileage = listing.find_elements(By.TAG_NAME, 'span')[1].text.strip()

    # Print the extracted information
    print(f"Title: {title}")
    print(f"Price: {price}")
    print(f"Mileage: {mileage}")
    print()

# Quit the browser
driver.quit()

NoSuchDriverException: Message: Unable to locate or obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
